In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import anndata
import string
import gc
from anndata import read_h5ad
from anndata import read_csv
from pandas import DataFrame
import h5py
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import networkx as nx
import plotly.graph_objects as go
import scanpy as sc
import pickle

/home/anton.thieme/miniconda3/envs/sc_env/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)


In [4]:
motif_df = pd.read_csv(os.path.join('..', 'results', 'motif', 'motif_sequence_data.csv'))

# filter for human
motif_df = motif_df[motif_df['Species'] == 'Homo sapiens']

In [3]:
tf_df = pd.read_csv(os.path.join('..', 'data', 'HumanTranscriptionFactorsEnsembl.csv'))
adata_tf = read_h5ad(os.path.join('..', 'data', 'adata_tf.h5ad'))
print(f'There are {len(list(adata_tf.var['gene_symbol'].unique()))} transcription factors in Tabula Sapiens')
print(f'There are {len(list(motif_df['Gene Name'].unique()))} motifs for human transcription factors')

tf_with_motif = list(set(list(adata_tf.var['gene_symbol'].unique())) &  set(list(motif_df['Gene Name'].unique())))
print(f'There are motifs for {len(tf_with_motif)} transcription factors in Tabula Sapiens')

There are 1637 transcription factors in Tabula Sapiens
There are 708 motifs for human transcription factors
There are motifs for 646 transcription factors in Tabula Sapiens


In [8]:
motif_df[motif_df['Gene Name'].isin(tf_with_motif)].drop_duplicates(subset='Gene Name', keep='first')

,AA Sequence,Collection,Gene Name,Matrix ID,Motif ID,Name,Species,Taxonomy ID,UniProt ID,pwm,pfm
1,MAAAAAALSGAGTPPAGGGAGGGGAGGGGSPPGGWAVARLEGREFE...,CORE,FOXK2,MA1103.1,MA1103.1,FOXK2,Homo sapiens,9606,Q01167,"[[-1.6370375651769136, -1.403681114886262, -1....","[[3349.0, 1353.0, 1566.0, 1709.0], [2030.0, 13..."
4,MAAAAGGAASRRGAGRPCPFSIEHILSSLPERSLPARAACPPQPAG...,CORE,GSC2,MA0891.1,MA0891.1,GSC2,Homo sapiens,9606,O15499,"[[-1.7107356393524489, -1.3722841508983814, -0...","[[461.0, 686.0, 442.0, 427.0], [356.0, 839.0, ..."
6,MAAAAPRRPTQQGTVTFEDVAVNFSQEEWCLLSEAQRCLYRDVMLE...,CORE,ZNF417,MA1727.1,MA1727.1,ZNF417,Homo sapiens,9606,Q8TAU3,"[[-1.399065408664144, -1.881914329790825, -2.3...","[[300.898, 324.85, 235.03, 139.222], [426.647,..."
8,MAAAAYVDHFAAECLVSMSSRAVVHGPREGPESRPEGAAVAATPTL...,CORE,KLF13,MA0657.1,MA0657.1,KLF13,Homo sapiens,9606,Q9Y2Y9,"[[-0.6394237240594488, -3.5579736982663066, -1...","[[1508.0, 248.0, 915.0, 590.0], [733.0, 46.0, ..."
10,MAAAKAEMQLMSPLQISDPFGSFPHSPTMDNYPKLEEMMLLSNGAP...,CORE,EGR1,MA0162.2,MA0162.2,EGR1,Homo sapiens,9606,P18146,"[[-1.9543649225491675, -2.244199227293854, -0....","[[1098.0, 5728.0, 3083.0, 2347.0], [1506.0, 68..."
...,...,...,...,...,...,...,...,...,...,...,...
1886,MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,CORE,NFIX,MA0671.1,MA0671.1,NFIX,Homo sapiens,9606,Q14938,"[[-1.6513445918815692, -1.533051659058266, -1....","[[7951.0, 8810.0, 8725.0, 7378.0], [7832.0, 61..."
1890,MYSSPLCLTQDEFHPFIEALLPHVRAFAYTWFNLQARKRKYFKKHE...,CORE,NFIC,MA0161.1,MA0161.1,NFIC,Homo sapiens,9606,P08651,"[[-1.8217863834767984, -0.1901989906071733, -4...","[[358.0, 1832.0, 176.0, 4546.0], [81.0, 67.0, ..."
1896,MYVGYVLDKDSPVYPGPARPASLGLGPQAYGPPAPPPAPPQYPDFS...,CORE,CDX1,MA0878.1,MA0878.1,CDX1,Homo sapiens,9606,P47902,"[[-2.438841348752184, -1.0994193071730278, 1.3...","[[2236.0, 1557.0, 4911.0, 570.0], [289.0, 5143..."
1899,MYVSYLLDKDVSMYPSSVRHSGGLNLAPQNFVSPPQYPDYGGYHVA...,CORE,CDX2,MA0465.1,MA0465.1,CDX2,Homo sapiens,9606,Q99626,"[[-1.9590309648937452, -0.9550236522171422, -0...","[[613.0, 319.0, 480.0, 185.0], [665.0, 8.0, 50..."


In [7]:
motif_df

,AA Sequence,Collection,Gene Name,Matrix ID,Motif ID,Name,Species,Taxonomy ID,UniProt ID,pwm,pfm
1,MAAAAAALSGAGTPPAGGGAGGGGAGGGGSPPGGWAVARLEGREFE...,CORE,FOXK2,MA1103.1,MA1103.1,FOXK2,Homo sapiens,9606,Q01167,"[[-1.6370375651769136, -1.403681114886262, -1....","[[3349.0, 1353.0, 1566.0, 1709.0], [2030.0, 13..."
2,MAAAAAALSGAGTPPAGGGAGGGGAGGGGSPPGGWAVARLEGREFE...,CORE,FOXK2,MA1103.2,MA1103.2,FOXK2,Homo sapiens,9606,Q01167,"[[-1.7006512423937714, -1.3684622135834574, -1...","[[6548.0, 2872.0, 3471.0, 3788.0], [4861.0, 21..."
3,MAAAAAALSGAGTPPAGGGAGGGGAGGGGSPPGGWAVARLEGREFE...,CORE,FOXK2,MA1103.3,MA1103.3,FOXK2,Homo sapiens,9606,Q01167,"[[-2.560074569959817, -2.922790958470947, 1.80...","[[2697.0, 551.0, 12737.0, 694.0], [360.0, 210...."
4,MAAAAGGAASRRGAGRPCPFSIEHILSSLPERSLPARAACPPQPAG...,CORE,GSC2,MA0891.1,MA0891.1,GSC2,Homo sapiens,9606,O15499,"[[-1.7107356393524489, -1.3722841508983814, -0...","[[461.0, 686.0, 442.0, 427.0], [356.0, 839.0, ..."
5,MAAAAGGAASRRGAGRPCPFSIEHILSSLPERSLPARAACPPQPAG...,CORE,GSC2,MA0891.2,MA0891.2,GSC2,Homo sapiens,9606,O15499,"[[-3.2487993723115065, -7.965784284662087, -7....","[[116.0, 0.0, 0.0, 2016.0], [2016.0, 87.0, 56...."
...,...,...,...,...,...,...,...,...,...,...,...
1898,MYVGYVLDKDSPVYPGPARPASLGLGPQAYGPPAPPPAPPQYPDFS...,CORE,CDX1,MA0878.3,MA0878.3,CDX1,Homo sapiens,9606,P47902,"[[-3.40430594872012, -1.6763421117595223, 0.84...","[[1088.0, 884.0, 7528.0, 1076.0], [527.0, 355...."
1899,MYVSYLLDKDVSMYPSSVRHSGGLNLAPQNFVSPPQYPDYGGYHVA...,CORE,CDX2,MA0465.1,MA0465.1,CDX2,Homo sapiens,9606,Q99626,"[[-1.9590309648937452, -0.9550236522171422, -0...","[[613.0, 319.0, 480.0, 185.0], [665.0, 8.0, 50..."
1900,MYVSYLLDKDVSMYPSSVRHSGGLNLAPQNFVSPPQYPDYGGYHVA...,CORE,CDX2,MA0465.2,MA0465.2,CDX2,Homo sapiens,9606,Q99626,"[[-2.10013969549322, -1.1124038050523877, -0.9...","[[11369.0, 6380.0, 7434.0, 6843.0], [11735.0, ..."
1901,MYVSYLLDKDVSMYPSSVRHSGGLNLAPQNFVSPPQYPDYGGYHVA...,CORE,CDX2,MA0465.3,MA0465.3,CDX2,Homo sapiens,9606,Q99626,"[[-3.5154055732855194, -2.0175380905408216, 1....","[[3157.0, 2096.0, 24214.0, 2559.0], [1226.0, 2..."
